In [1]:
# 1) Install required packages
%pip install -qU arxiv langgraph langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 2) Define external search tools (Wikipedia, Arxiv)
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250))
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250))

print("Defined tools: wiki, arxiv")

Defined tools: wiki, arxiv


In [3]:
# 3) Create knowledge base and retriever (LangSmith docs) using Ollama embeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Building vector store (requires Ollama running)...")
try:
    loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = splitter.split_documents(docs)
    embeddings = OllamaEmbeddings(model="llama3.1:8b")
    vectordb = FAISS.from_documents(documents, embeddings)
    retriever = vectordb.as_retriever()
    print("Retriever created.")
except Exception as e:
    retriever = None
    print(f"Retriever setup failed: {e}")
    print("Start Ollama with `ollama serve` and pull the model `ollama pull llama3.1:8b`.")

/Users/aryamanagarwal/Desktop/vs code Aryaman/langChain/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


Building vector store (requires Ollama running)...


/var/folders/bk/l5jscvds63s4hht82gtm_7j40000gn/T/ipykernel_10834/3910506140.py:13: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.1:8b")


Retriever created.


In [4]:
# 4) Define custom LangSmith search tool using the retriever
from langchain_core.tools import tool

@tool
def langsmith_search(query: str) -> str:
    """Search the LangSmith documentation for information."""
    if retriever is None:
        return "Retriever not available. Start Ollama and re-run cell 3."
    docs = retriever.invoke(query)
    return "\n\n".join(d.page_content for d in docs)

print("Defined tool: langsmith_search")

Defined tool: langsmith_search


In [5]:
# 5) Assemble all tools
tools = [wiki, arxiv, langsmith_search]
print("Unified tool list ready:", [t.name for t in tools])

Unified tool list ready: ['wikipedia', 'arxiv', 'langsmith_search']


In [ ]:
# 6) Choose and configure the LLM (Groq default; switch to Ollama if you prefer)
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOllama

load_dotenv()

# Read your Groq API key from environment or a local .env file
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "")

LLM_CHOICE = "groq"  # change to "ollama" if you have Ollama running locally
print(f"Using LLM: {LLM_CHOICE}")

llm = None
if LLM_CHOICE == "groq":
    if not GROQ_API_KEY:
        raise ValueError("GROQ_API_KEY is empty; set it in your environment or .env file.")
    llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0, groq_api_key=GROQ_API_KEY)
else:
    try:
        llm = ChatOllama(model="llama3.1:8b", temperature=0)
        llm.invoke("hi")
    except Exception as e:
        print(f"Ollama not available: {e}")
        llm = None

Using LLM: groq


In [20]:
# 7) Create the LangGraph ReAct agent
from langgraph.prebuilt import create_react_agent

# Use the built-in default prompt/schema for the ReAct agent
if llm:
    agent_executor = create_react_agent(llm, tools)
    print("LangGraph agent created.")
else:
    agent_executor = None
    print("LLM not available; agent not created.")

LangGraph agent created.


/var/folders/bk/l5jscvds63s4hht82gtm_7j40000gn/T/ipykernel_10834/3645606657.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)


In [21]:
# 8) Run the agent on a sample question (with a higher recursion limit)
if agent_executor:
    question = "What is machine learning?"
    print(f"Asking: {question}\n---")
    try:
        for idx, chunk in enumerate(
            agent_executor.stream(
                {"messages": [("user", question)]},
                config={"recursion_limit": 50},  # allow more steps before halting
            ),
            start=1,
        ):
            print(f"Step {idx}: {chunk}")
            print("----")
    except Exception as e:
        print(f"Agent run failed: {e}")
else:
    print("Agent not available; check LLM/retriever setup.")

Asking: What is machine learning?
---
Step 1: {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qwqcgy1fb', 'function': {'arguments': '{"query":"machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 490, 'total_tokens': 505, 'completion_time': 0.039538118, 'completion_tokens_details': None, 'prompt_time': 0.035967839, 'prompt_tokens_details': None, 'queue_time': 0.06438496, 'total_time': 0.075505957}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b2728-cf00-7ca3-9e33-e5e95070bfd1-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'machine learning'}, 'id': 'qwqcgy1fb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 490, 'output_tokens': 15, 'total_tokens': 505})]}}
----
Step 2: {'tools': {'messages':

KeyboardInterrupt: 